In [1]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt

In [7]:
os.listdir()
os.path.join(os.getcwd(), 'train')

'D:\\Document\\bangkit\\ktpocr\\train'

In [8]:
base_dir = os.path.join(os.getcwd(), 'train')
IMAGE_SIZE = 224
BATCH_SIZE = 5
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1. / 255,
    validation_split=0.2)

In [9]:
train_generator = data_generator.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='training')

val_generator = data_generator.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation')

for image_batch, label_batch in train_generator:
    break

Found 169 images belonging to 3 classes.
Found 41 images belonging to 3 classes.


In [10]:
print(train_generator.class_indices)
labels = '\n'.join(sorted(train_generator.class_indices.keys()))
with open('labels.txt', 'w') as f:
    f.write(labels)

{'farhan': 0, 'tasya': 1, 'wesli': 2}


In [11]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')


9420800/9406464 [==============================] - 2s 0us/step


In [14]:
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

 block_3_expand_relu (ReLU)     (None, 56, 56, 144)  0           ['block_3_expand_BN[0][0]']      
                                                                                                  
 block_3_pad (ZeroPadding2D)    (None, 57, 57, 144)  0           ['block_3_expand_relu[0][0]']    
                                                                                                  
 block_3_depthwise (DepthwiseCo  (None, 28, 28, 144)  1296       ['block_3_pad[0][0]']            
 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 28, 28, 144)  576        ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_d

 lization)                                                                                        
                                                                                                  
 block_7_expand (Conv2D)        (None, 14, 14, 384)  24576       ['block_6_project_BN[0][0]']     
                                                                                                  
 block_7_expand_BN (BatchNormal  (None, 14, 14, 384)  1536       ['block_7_expand[0][0]']         
 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 14, 14, 384)  0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 14, 14, 384)  3456       ['block_7_expand_relu[0][0]']    
 nv2D)    

                                                                                                  
 block_10_depthwise_relu (ReLU)  (None, 14, 14, 384)  0          ['block_10_depthwise_BN[0][0]']  
                                                                                                  
 block_10_project (Conv2D)      (None, 14, 14, 96)   36864       ['block_10_depthwise_relu[0][0]']
                                                                                                  
 block_10_project_BN (BatchNorm  (None, 14, 14, 96)  384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 14, 14, 576)  55296       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_

 block_14_depthwise (DepthwiseC  (None, 7, 7, 960)   8640        ['block_14_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_14_depthwise_BN (BatchNo  (None, 7, 7, 960)   3840        ['block_14_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 7, 7, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 7, 7, 160)    153600      ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_

In [13]:
#biar yg di train cuma yang tambahan
base_model.trainable = False

In [15]:
model = tf.keras.Sequential([
    base_model,  # 1
    tf.keras.layers.Conv2D(32, 3, activation='relu'),  # 2
    tf.keras.layers.Dropout(0.2),  # 3
    tf.keras.layers.GlobalAveragePooling2D(),  # 4
    tf.keras.layers.Dense(3, activation='softmax')  # 5
])

In [16]:

model.compile(optimizer=tf.keras.optimizers.Adam(),  
              loss='categorical_crossentropy', 
              metrics=['accuracy'])  


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 conv2d (Conv2D)             (None, 5, 5, 32)          368672    
                                                                 
 dropout (Dropout)           (None, 5, 5, 32)          0         
                                                                 
 global_average_pooling2d (G  (None, 32)               0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 3)                 99        
                                                                 
Total params: 2,626,755
Trainable params: 368,771
Non-tr

In [17]:
# Printing some statistics
print('Number of trainable variables = {}'.format(len(model.trainable_variables)))

# Train the model
# We will do it in 10 Iterations
epochs = 10

# Fitting / Training the model
history = model.fit(train_generator,
                    epochs=epochs,
                    validation_data=val_generator)

Number of trainable variables = 4
Epoch 1/10
34/34 [==============================] - 32s 419ms/step - loss: 0.1484 - accuracy: 0.9408 - val_loss: 3.7056e-05 - val_accuracy: 1.0000
Epoch 2/10
34/34 [==============================] - 4s 109ms/step - loss: 1.9406e-05 - accuracy: 1.0000 - val_loss: 5.5737e-06 - val_accuracy: 1.0000
Epoch 3/10
34/34 [==============================] - 3s 99ms/step - loss: 5.8312e-06 - accuracy: 1.0000 - val_loss: 4.8933e-06 - val_accuracy: 1.0000
Epoch 4/10
34/34 [==============================] - 3s 97ms/step - loss: 7.6686e-06 - accuracy: 1.0000 - val_loss: 4.3700e-06 - val_accuracy: 1.0000
Epoch 5/10
34/34 [==============================] - 3s 97ms/step - loss: 6.5359e-06 - accuracy: 1.0000 - val_loss: 3.9513e-06 - val_accuracy: 1.0000
Epoch 6/10
34/34 [==============================] - 3s 95ms/step - loss: 6.1380e-06 - accuracy: 1.0000 - val_loss: 3.4541e-06 - val_accuracy: 1.0000
Epoch 7/10
34/34 [==============================] - 4s 107ms/step - loss:

In [31]:
import numpy as np
from keras.preprocessing import image

img = image.load_img(os.path.join(os.getcwd(),'train','wesli','wesli1.jpeg'), target_size=(224,224))
x = image.img_to_array(img)
x /= 255
x = np.expand_dims(x, axis=0)

In [32]:
model.predict(x)

array([[4.8169624e-15, 4.6525991e-15, 1.0000000e+00]], dtype=float32)

In [35]:
model.save(os.path.join(os.getcwd(),'save','verifktp.h5'))